In [235]:
import load_data
import os
from netCDF4 import Dataset

In [3]:
import pandas as pd
import numpy as np

In [326]:
obs_names = ['psur_obs', 't2m_obs', 'q2m_obs', 'w10m_obs', 'd10m_obs', 'rh2m_obs', 'u10m_obs', 'v10m_obs', 'RAIN_obs']
m_names = ['q2m_M', 'rh2m_M', 'w10m_M', 'd10m_M', 'u10m_M', 'v10m_M', 'SWD_M', 'GLW_M', 'HFX_M',
           'LH_M', 'RAIN_M', 'PBLH_M', 'TC975_M', 'TC925_M', 'TC850_M', 'TC700_M', 'TC500_M', 'wspd975_M', 'wspd925_M',
           'wspd850_M', 'wspd700_M', 'wspd500_M', 'Q975_M', 'Q925_M', 'Q850_M', 'Q700_M', 'Q500_M']

In [304]:
explain = pd.read_csv(os.path.join('data', 'explain.csv'), index_col=0).dropna(how='any', axis=1)

In [246]:
def get_serial_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fetch data with serial datetime.
    """
    df_series = pd.DataFrame()
    for _, group in df.groupby(['station_id', 'date']):
        df_series = pd.concat([df_series, group.iloc[:24]])
    date_index = pd.Series()
    for station_id, group in df_series.groupby('station_id'):
        index = pd.date_range(group.loc[station_id].index[0][0], group.loc[station_id].index[-1][0] + pd.DateOffset(hours=23), freq='H')
        index = index.to_series()
        date_index = pd.concat([date_index, index])
    df_series.index = df_series.index.droplevel([1, 2])
    df_series = df_series.set_index(date_index, append=True)
    return df_series

In [238]:
def load_raw_file(file_dir: str) -> pd.DataFrame:
    """
    Load data from original netCDF4 file.

    :return: one pd.DataFrame containing all data in file.
    """
    print("Loading netCDF4 file", file_dir)
    data = Dataset(file_dir)
    date_list = [pd.Timestamp.strptime(str(int(this_date)), '%Y%m%d%H') for this_date in data['date']]
    station_list = list(data['station'][:])
    feature_list = list(data.variables.keys())[3:]

    date_index = []
    for date in date_list:
        date_index += [date] * data['foretimes'][:].shape[0]
    foretime_index = list(data['foretimes'][:]) * data['date'][:].shape[0]

    df_list = []
    for i in range(len(station_list)):
        print("Loading station", station_list[i])
        df = pd.DataFrame()
        df['date'] = date_index
        df['foretime'] = foretime_index
        for feature in feature_list:
            feature_df = pd.DataFrame(data[feature][:, :, i].reshape(-1, 1), columns=[feature])
            df = pd.concat([df, feature_df], axis=1)
        df['station_id'] = station_list[i]
        df_list.append(df)
    result = pd.concat(df_list)
    result = result.set_index(['station_id', 'date', 'foretime'])
    return result

In [239]:
tr_raw = load_raw_file(os.path.join('data', 'ai_challenger_wf2018_trainingset_20150301-20180531.nc'))

Loading netCDF4 file data/ai_challenger_wf2018_trainingset_20150301-20180531.nc
Loading station 90001
Loading station 90002
Loading station 90003
Loading station 90004
Loading station 90005
Loading station 90006
Loading station 90007
Loading station 90008
Loading station 90009
Loading station 90010


In [240]:
va_raw = load_raw_file(os.path.join('data', 'ai_challenger_wf2018_validation_20180601-20180828_20180905.nc'))

Loading netCDF4 file data/ai_challenger_wf2018_validation_20180601-20180828_20180905.nc
Loading station 90001
Loading station 90002
Loading station 90003
Loading station 90004
Loading station 90005
Loading station 90006
Loading station 90007
Loading station 90008
Loading station 90009
Loading station 90010


In [15]:
def fill_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fill data using basic linear insertion.
    """
    return df.groupby(level=[0, 1]).apply(lambda group: group.interpolate(method='linear', limit=3, 
                                                                          limit_direction='both'))

In [6]:
def drop_empty_day(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop days where there is empty data.
    """
    for name in df.columns:
        empty_index_list = list(df.loc[df[name].apply(np.isnan)].groupby(['station_id', 'date']).groups.keys())
        if len(empty_index_list) > 0:
            df = df.drop(index=empty_index_list)
    return df

In [308]:
def normalize(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize data.
    """
    result = pd.DataFrame(df)
    for name in df.columns:
        scope = explain.loc[name]['scope'][1:-1].split(',')
        min_value = float(scope[0].strip())
        max_value = float(scope[1].strip())
        
        result[name] = (result[name] - min_value) / (max_value - min_value)
    return result

In [7]:
tr_fill = fill_data(tr_raw)

In [241]:
va_fill = fill_data(va_raw)

In [8]:
tr_drop = drop_empty_day(tr_fill)

In [242]:
va_drop = drop_empty_day(va_fill)

In [398]:
tr_n = normalize(tr_drop)

In [321]:
tr_fill

psfc_M     t2m_M     q2m_M  \
station_id date                foretime                                 
90001      2015-03-01 03:00:00 0              NaN       NaN       NaN   
                               1              NaN       NaN       NaN   
                               2              NaN       NaN       NaN   
                               3              NaN       NaN       NaN   
                               4              NaN       NaN       NaN   
                               5              NaN       NaN       NaN   
                               6              NaN       NaN       NaN   
                               7              NaN       NaN       NaN   
                               8              NaN       NaN       NaN   
                               9              NaN       NaN       NaN   
                               10             NaN       NaN       NaN   
                               11             NaN       NaN       NaN   
                               12             NaN       NaN       NaN   
                               13             NaN       NaN       NaN   
                               14             NaN       NaN       NaN   
                               15             NaN       NaN       NaN   
                               16             NaN       NaN       NaN   
                               17             NaN       NaN       NaN   
                               18             NaN       NaN       NaN   
                               19             NaN       NaN       NaN   
                               20             NaN       NaN       NaN   
                               21             NaN       NaN       NaN   
                               22             NaN       NaN       NaN   
                               23             NaN       NaN       NaN   
                               24             NaN       NaN       NaN   
                               25             NaN       NaN       NaN   
                               26             NaN       NaN       NaN   
                               27             NaN       NaN       NaN   
                               28             NaN       NaN       NaN   
                               29             NaN       NaN       NaN   
...                                           ...       ...       ...   
90010      2018-05-31 03:00:00 7        -3.413654  0.425532  0.000007   
                               8        -3.413654  0.425532  0.000008   
                               9        -3.413654  0.425532  0.000006   
                               10       -3.413654  0.425532  0.000006   
                               11       -3.413654  0.425532  0.000006   
                               12       -3.413654  0.425532  0.000006   
                               13       -3.413654  0.425532  0.000006   
                               14       -3.413654  0.425532  0.000006   
                               15       -3.413654  0.425532  0.000006   
                               16       -3.413654  0.425532  0.000006   
                               17       -3.413654  0.425532  0.000006   
                               18       -3.413654  0.425532  0.000006   
                               19       -3.413654  0.425532  0.000008   
                               20       -3.413654  0.425532  0.000006   
                               21       -3.413654  0.425532  0.000006   
                               22       -3.413654  0.425532  0.000006   
                               23       -3.413654  0.425532  0.000006   
                               24       -3.413654  0.425532  0.000006   
                               25       -3.413654  0.425532  0.000005   
                               26       -3.413654  0.425532  0.000005   
                               27       -3.413654  0.425532  0.000005   
                               28       -3.413654  0.425532  0.000005   
                            

In [310]:
tr_n

psfc_M     t2m_M         q2m_M  \
station_id date                foretime                                     
90001      2015-03-01 03:00:00 0              NaN       NaN           NaN   
                               1              NaN       NaN           NaN   
                               2              NaN       NaN           NaN   
                               3              NaN       NaN           NaN   
                               4              NaN       NaN           NaN   
                               5              NaN       NaN           NaN   
                               6              NaN       NaN           NaN   
                               7              NaN       NaN           NaN   
                               8              NaN       NaN           NaN   
                               9              NaN       NaN           NaN   
                               10             NaN       NaN           NaN   
                               11             NaN       NaN           NaN   
                               12             NaN       NaN           NaN   
                               13             NaN       NaN           NaN   
                               14             NaN       NaN           NaN   
                               15             NaN       NaN           NaN   
                               16             NaN       NaN           NaN   
                               17             NaN       NaN           NaN   
                               18             NaN       NaN           NaN   
                               19             NaN       NaN           NaN   
                               20             NaN       NaN           NaN   
                               21             NaN       NaN           NaN   
                               22             NaN       NaN           NaN   
                               23             NaN       NaN           NaN   
                               24             NaN       NaN           NaN   
                               25             NaN       NaN           NaN   
                               26             NaN       NaN           NaN   
                               27             NaN       NaN           NaN   
                               28             NaN       NaN           NaN   
                               29             NaN       NaN           NaN   
...                                           ...       ...           ...   
90010      2018-05-31 03:00:00 7        -3.413655  0.425532  2.474263e-07   
                               8        -3.413655  0.425532  2.711529e-07   
                               9        -3.413655  0.425532  2.072452e-07   
                               10       -3.413655  0.425532  1.856560e-07   
                               11       -3.413655  0.425532  1.899402e-07   
                               12       -3.413655  0.425532  2.004250e-07   
                               13       -3.413655  0.425532  2.053015e-07   
                               14       -3.413655  0.425532  2.006207e-07   
                               15       -3.413655  0.425532  1.970656e-07   
                               16       -3.413655  0.425532  1.959588e-07   
                               17       -3.413655  0.425532  1.921594e-07   
                               18       -3.413655  0.425532  1.884642e-07   
                               19       -3.413655  0.425532  2.563239e-07   
                               20       -3.413655  0.425532  2.158470e-07   
                               21       -3.413655  0.425532  2.133612e-07   
                               22       -3.413655  0.425532  2.004613e-07   
                               23       -3.413655  0.425532  1.958549e-07   
                               24       -3.413655  0.425532  1.886211e-07   
                               25       -3.413655  0.425532  1.818045e-07   
                               26      

In [9]:
def check_empty_day(df: pd.DataFrame) -> pd.DataFrame:
    result = set()
    for name in df.columns:
        empty_index_set = set(df.loc[df[name].apply(np.isnan)].groupby(['station_id', 'date']).groups.keys())
        result |= empty_index_set
    result = pd.DataFrame(list(result), columns=['station_id', 'date'])
    result = result.sort_values(by=['station_id', 'date'])
    result = result.set_index('station_id')
    return result

In [10]:
all_empty = check_empty_day(tr_fill)

In [11]:
all_empty.to_csv('empty_dates.csv')
all_empty.to_pickle('empty_dates.pkl')

In [12]:
tr_fill.to_pickle('tr_fill.pkl')

In [13]:
tr_drop.to_pickle('tr_drop.pkl')

In [49]:
print(m_names)

['psfc_M', 't2m_M', 'q2m_M', 'rh2m_M', 'w10m_M', 'd10m_M', 'u10m_M', 'v10m_M', 'SWD_M', 'GLW_M', 'HFX_M', 'LH_M', 'RAIN_M', 'PBLH_M', 'TC975_M', 'TC925_M', 'TC850_M', 'TC700_M', 'TC500_M', 'wspd975_M', 'wspd925_M', 'wspd850_M', 'wspd700_M', 'wspd500_M', 'Q975_M', 'Q925_M', 'Q850_M', 'Q700_M', 'Q500_M']


In [212]:
def generate_one_set(station_id: int, forecast_date: pd.Timestamp, df: pd.DataFrame, previous_days=1):
    """
    Use forecast date's data and previous date data to concat a set of training data.
    """
    history_list = []
    for i in range(previous_days, 0, -1):
        history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
    history = pd.concat(history_list + [df.loc[station_id, forecast_date]])

    history_obs = history.iloc[:(24*previous_days + 4)][obs_names]
    history_m = history[m_names]

    prediction = history.iloc[(24*previous_days + 4):][['t2m_obs', 'rh2m_obs', 'w10m_obs']]

    return history_obs, history_m, prediction

In [219]:
try:
    a, b, c = generate_one_set(90001, pd.Timestamp(2015, 3, 27, 3), tr_drop, previous_days=2)
except KeyError:
    print("Contains empty data.")

In [224]:
def generate_history_obs_data(obs_df: pd.DataFrame) -> pd.Series:
    """
    Use observation history data frame generated by function generate_one_set() 
    to construct training data.
    """
    return generate_series_data(obs_df, len(obs_names))

In [190]:
def generate_history_m_data(m_df: pd.DataFrame) -> pd.Series:
    """
    Use Ruitu history data frame genrated by function generate_one_set()
    to construct training data.
    """
    return generate_series_data(m_df, len(m_names))

In [206]:
def generate_series_data(df: pd.DataFrame, column_length: int) -> pd.Series:
    df_max = df.max()
    df_max.index = df_max.index + '_max'
    df_min = df.min()
    df_min.index = df_min.index + '_min'
    df_mean = df.mean()
    df_mean.index = df_mean.index + '_mean'
    df_var = df.var()
    df_var.index = df_var.index + '_var'
    
    df_series = df.stack()
    series_index = np.array([[i] * column_length for i in range(int(len(df_series.index) / column_length))]).reshape(1, -1)[0].astype(str)
    df_series.index = df_series.index.get_level_values(1) + '_' + series_index
    
    return pd.concat([df_series, df_max, df_min, df_mean, df_var])

In [111]:
holiday = ['2017-01-01', '2017-01-02', '2017-01-27', '2017-01-28', '2017-01-29',
           '2017-01-30', '2017-01-31', '2017-02-01', '2017-02-02', '2017-04-02',
           '2017-04-03', '2017-04-04', '2017-04-29', '2017-04-30', '2017-05-01',
           '2017-05-28', '2017-05-29', '2017-05-30', '2017-10-01', '2017-10-02',
           '2017-10-03', '2017-10-04', '2017-10-05', '2017-10-06', '2017-10-07',
           '2017-10-08', '2017-12-30', '2017-12-31',
           '2018-01-01', '2018-02-15', '2018-02-16', '2018-02-17', '2018-02-18',
           '2018-02-19', '2018-02-20', '2018-02-21', '2018-04-05', '2018-04-06',
           '2018-04-07', '2018-04-29', '2018-04-30', '2018-05-01', '2018-06-16',
           '2018-06-17', '2018-06-18']
work = ['2017-01-22', '2017-02-04', '2017-04-01', '2017-05-27', '2017-09-30',
        '2018-02-11', '2018-02-24', '2018-04-08', '2018-04-28']

In [367]:
def generate_date_feature(date: pd.Timestamp):
    """
    Generate date features.
    """
    result = pd.Series()
    result['timestamp'] = date.timestamp()
    
    format_string = '%Y-%m-%d'
    dt_string = date.strftime(format_string)
    result['holiday'] = int((dt_string in holiday) or (date.weekday() in [5, 6] and dt_string not in work))
    
    result = pd.concat([result, get_onehot(date.weekday(), 0, 6, name='weekday')])
    return result

In [251]:
def generate_stat_feature(station_id: int, forecast_date: pd.Timestamp, df: pd.DataFrame, days):
    """
    Using assigned length of history data to fetch statistic features.
    """
    history_list = []
    for i in range(days, 0, -1):
        try:
            history_list.append(df.loc[station_id, forecast_date - pd.DateOffset(days=i)].iloc[:24])
        except KeyError:
            pass
    h = pd.concat(history_list + [df.loc[station_id, forecast_date]])
    
    h_max = h.max()
    h_min = h.min()
    h_mean = h.mean()
    h_var = h.var()
    
    h_max.index = h_max.index + '_{}days_max'.format(days)
    h_min.index = h_min.index + '_{}days_min'.format(days)
    h_mean.index = h_mean.index + '_{}days_mean'.format(days)
    h_var.index = h_var.index + '_{}days_var'.format(days)
    
    return pd.concat([h_max, h_min, h_mean, h_var])

In [373]:
def get_onehot(value, min_value, max_value, step=1, name="") -> pd.Series:
    length = int((max_value - min_value) / step) + 1
    result = np.zeros([length])
    index = int((value - min_value) / step)
    result[index] = 1
    
    name_list = [name + "_{}".format(i) for i in range(length)]
    result = pd.Series(result, index=name_list)
    return result

In [387]:
def generate_x(station_id: int, forecast_date: pd.Timestamp, df, days=1):
    history_obs, history_m, prediction = generate_one_set(station_id, forecast_date, df, days)
    obs_s = generate_history_obs_data(history_obs)
    m_s = generate_history_m_data(history_m)
    date_f = generate_date_feature(forecast_date)
    week_s = generate_stat_feature(station_id, forecast_date, df, 7)
    month_s = generate_stat_feature(station_id, forecast_date, df, 30)
    
    return pd.concat([obs_s, m_s, date_f, week_s, month_s]), prediction

In [318]:
def denormalize(df: pd.DataFrame):
    result = pd.DataFrame(df)
    for name in df.columns:
        scope = explain.loc[name]['scope'][1:-1].split(',')
        min_value = float(scope[0].strip())
        max_value = float(scope[1].strip())
        
        result[name] = result[name] * (max_value - min_value) + min_value
    return result

In [399]:
X_1, Y_1 = generate_x(90001, pd.Timestamp(2015, 3, 27, 3), tr_n, 2)
X_2, Y_2 = generate_x(90001, pd.Timestamp(2015, 3, 28, 3), tr_n, 2)

In [400]:
pd.concat([X_1, X_2], axis=1).transpose()

,psur_obs_0,t2m_obs_0,q2m_obs_0,w10m_obs_0,d10m_obs_0,rh2m_obs_0,u10m_obs_0,v10m_obs_0,RAIN_obs_0,psur_obs_1,...,Q500_M_30days_var,psur_obs_30days_var,t2m_obs_30days_var,q2m_obs_30days_var,w10m_obs_30days_var,d10m_obs_30days_var,rh2m_obs_30days_var,u10m_obs_30days_var,v10m_obs_30days_var,RAIN_obs_30days_var
0,0.7140,0.549474,0.092118,0.060000,0.622222,0.32,0.520840,0.521580,0.0,0.7116,...,0.000117,0.000708,0.004668,0.001659,0.003001,0.086213,0.039746,0.000913,0.001363,0.0
1,0.7064,0.592632,0.117076,0.056667,0.725000,0.31,0.527984,0.504432,0.0,0.7044,...,0.000132,0.000714,0.004873,0.001762,0.002978,0.084333,0.038904,0.000930,0.001357,0.0
